<a id="section-one"></a>
## Intro + Setup

* [Intro + Setup](#section-one)
* [Data Cleaning](#section-two)
* [Data Viz](#section-three)
* [Conclusion](#section-four)

The purpose of this notebook is to make notes about the use of deadly force by Rio de Janeiro State's police by ethnicity, specially distinguishing whites from blacks and pardos (mixed-race brazilians, often lightskin afro-brazilians).

I used a dataset from Rio's Public Safety Institute which contains crimes that occurred between 2009 and 2019.

---

O objetivo deste notebook é fazer observações sobre a letalidade policial no Estado do Rio de Janeiro, adotando um recorte étnico para comparar a incidência do uso de força letal por parte da polícia entre vítimas brancas e negras (pardas e pretas). 

A base de dados utilizada na análise foi disponibilizada pelo Instituto de Segurança Pública do Rio de Janeiro, e contempla dados entre os anos de 2009 e 2019.

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
sns.set(style = 'whitegrid')

In [ ]:
import os
for dirname, _, filenames in os.walk('../input/crimes-estado-rio-de-janeiro-2009-a-2019/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<a id="section-two"></a>
## Data cleaning

In [ ]:
parte1 = pd.read_csv('../input/crimes-estado-rio-de-janeiro-2009-a-2019/parte1.csv', 
                     sep = ';', 
                     encoding = 'iso-8859-1')

parte1 = parte1.loc[parte1.titulo_do == 'Morte por intervenção de agente do Estado']

parte1.drop(['risp', 'aisp'], axis = 1, inplace = True)


parte2 = pd.read_csv('../input/crimes-estado-rio-de-janeiro-2009-a-2019/parte2.csv', 
                     sep = ';', 
                     encoding = 'iso-8859-1')

parte2 = parte2.loc[parte2.titulo_do == 'Morte por intervenção de agente do Estado']


parte3 = pd.read_csv('../input/crimes-estado-rio-de-janeiro-2009-a-2019/parte3.csv', 
                     sep = ';', 
                     encoding = 'iso-8859-1')

parte3 = parte3.loc[parte3.titulo_do == 'Morte por intervenção de agente do Estado']


parte4 = pd.read_csv('../input/crimes-estado-rio-de-janeiro-2009-a-2019/parte4.csv', 
                     sep = ';', 
                     encoding = 'iso-8859-1')

parte4 = parte4.loc[parte4.titulo_do == 'Morte por intervenção de agente do Estado']

mortes_estado = pd.concat([parte1, parte2, parte3, parte4])
mortes_estado

In [ ]:
serie_cisp_mortes_estado = pd.value_counts(mortes_estado['cisp'].values, sort = True)
cisp_mortes_estado = pd.DataFrame({'cisp' : serie_cisp_mortes_estado.index, 'mortos' : serie_cisp_mortes_estado.values})
pd.set_option('display.max_rows', None)
cisp_mortes_estado.set_index('cisp')

In [ ]:
serie_idade_mortes_estado = pd.value_counts(mortes_estado['idade'].values, sort = True)
idade_mortes_estado = pd.DataFrame({'idade' : serie_idade_mortes_estado.index, 'mortos' : serie_idade_mortes_estado.values})
pd.set_option('display.max_rows', None)
idade_mortes_estado.set_index('idade')

In [ ]:
serie_cor_mortes_estado = pd.value_counts(mortes_estado['cor'].values, sort = True)
cor_mortes_estado = pd.DataFrame({'cor' : serie_cor_mortes_estado.index, 'mortos' : serie_cor_mortes_estado.values})
pd.set_option('display.max_rows', None)
cor_mortes_estado.set_index('cor')

After setting up the workspace, printing the files and concatenating them (the whole data was divided because of the amount of data), I printed the df with the whole data and also some other dfs counting data entries (the "idade_mortes_estado" df shows that the majority of police lethality victims are young adults; and the "cisp_mortes_estado" df let us observe the number of deaths by occurrences in areas served by police stations) and to check the incorrect and unnecessary values as well.



The dataframe obtained after cleaning is down below.

---

Após importar as livrarias e unir os dados (que estavam em arquivos separados devido ao tamanho) em um só dataframe, printei o dataframe com a totalidade dos dados e também alguns outros pra contar as entradas (vê-se no dataframe "idade_mortes_estado", por exemplo, que a maior incidência de vítimas de violência policial é de jovens adultos; e o "cisp_mortes_estado permite observar as delegacias que registraram o maior nº de mortes via uso letal de força policial em sua área de atuação) e checar certos valores a fim de descartar entradas incorretas ou que não seriam úteis na análise. 

Abaixo, está o novo dataframe obtido a partir da filtragem dos dados.

In [ ]:
mortes_estado = mortes_estado[mortes_estado.cor != 'ignorado']
mortes_estado = mortes_estado[mortes_estado.cor != 'sem informação']
mortes_estado = mortes_estado[mortes_estado.cor != 'albino']
mortes_estado = mortes_estado[mortes_estado.cor != 'amarela']
mortes_estado = mortes_estado[mortes_estado.cor != 'índio']
mortes_estado = mortes_estado[mortes_estado.hora_fato != '99']
mortes_estado = mortes_estado[mortes_estado.idade != 118.0]
mortes_estado['cor'].replace({'negra': 'preta'}, inplace = True)
mortes_estado.reset_index(inplace = True)
mortes_estado.drop(['index', 'controle', 'dp'], axis = 1, inplace = True)
faixa_etaria = pd.cut(mortes_estado.idade, 
                   bins = [0, 17, 99], labels = ['menor de idade', 'maior de idade'])
mortes_estado.insert(21,'faixa etária', faixa_etaria)

mortes_estado['hora_fato'] = pd.to_datetime(mortes_estado['hora_fato'], format = '%H:%M')
mortes_estado = mortes_estado.assign(horário = pd.cut(mortes_estado.hora_fato.dt.hour,
                            [0, 6, 12, 18, 23],
                            labels = ['madrugada', 'manhã', 'tarde', 'noite'],
                            include_lowest = True))


reg = {'Zona Norte': ['039a. Pavuna', '021a. Bonsucesso', '038a. Braz de Pina', '025a. Engenho Novo', '040a. Honório Gurgel', '022a. Penha', '027a. Vicente de Carvalho', '029a. Madureira', '044a. Inhaúma', '031a. Ricardo Albuquerque', '037a. Ilha do Governador', '024a. Piedade', '030a. Marechal Hermes', '019a. Tijuca', '026a. Todos os Santos', '045a. Alemão', '020a. Grajaú', '028a. Campinho', '023a. Meier', '018a. Praça da Bandeira'], 
        'Zona Oeste': ['034a. Bangu', '036a. Santa Cruz', '032a. Taquara', '033a. Realengo', '035a. Campo Grande', '041a. Tanque', '016a. Barra da Tijuca', '042a. Recreio', '043a. Pedra de Guaratiba'], 
        'Zona Sul e Centro': ['006a. Cidade Nova', '017a. São Cristóvão', '007a. Santa Tereza', '004a. Praça da República', '011a. Rocinha', '015a. Gávea', '012a. Copacabana', '009a. Catete', '005a. Mem de Sá', '013a. Ipanema', '010a. Botafogo', '014a. Leblon'], 
        'Região Metropolitana (exceto capital)': ['054a. Belford Roxo', '059a. Duque de Caxias', '078a. Fonseca', '074a. Alcantara', '056a. Comendador Soares', '064a. Vilar dos Teles', '062a. Imbariê', '060a. Campos Elíseos', '073a. Neves', '072a. São Gonçalo', '050a. Itaguaí', '075a. Rio do Ouro', '071a. Itaboraí', '053a. Mesquita', '063a. Japerí', '077a. Icaraí', '079a. Jurujuba', '058a. Posse', '055a. Queimados', '076a. Niterói - Centro', '065a. Magé', '052a. Nova Iguaçu', '081a. Itaipú', '066a. Piabetá', '082a. Maricá', '057a. Nilópolis', '105a. Petrópolis', '048a. Seropédica', '051a. Paracambí', '067a. Guapimirim', '159a. Cachoeira de Macacú', '070a. Tanguá', '061a. Xerém', '119a. Rio Bonito', '106a. Itaipava'],
        'Interior': ['166a. Angra dos Reis', '123a. Macaé', '126a. Cabo Frio', '093a. Volta Redonda', '118a. Araruama', '151a. Nova Friburgo', '125a. São Pedro da Aldeia', '110a. Teresópolis', '146a. Guarus', '121a. Casimiro de Abreu', '124a. Saquarema', '090a. Barra Mansa', '134a. Campos', '128a. Rio das Ostras', '127a. Búzios', '167a. Parati', '165a. Mangaratiba', '089a. Resende', '132a. Arraial do Cabo', '099a. Itatiaia', '108a. Três Rios', '136a. Santo Antonio de Pádua', '147a. São Francisco de Itabapoana', '122a. Conceição de Macabú', '129a. Iguaba', '130a. Quissamã', '137a. Miracema', '138a. Lajes de Muriaré', '088a. Barra do Piraí', '145a. São João da Barra', '112a. Carmo', '139a. Porciúncula', '098a. Paulo de Frontin', '120a. Silva Jardim', '097a. Mendes', '143a. Itaperuna', '091a. Valença']}

dreg = {k: oldk for oldk, oldv in reg.items() for k in oldv}

mortes_estado['região'] = mortes_estado['cisp'].map(dreg)



mortes_estado

<a id="section-three"></a>
## Data Viz

First of all, I plotted the number of deaths by period of time (night, morning, afternoon and evening, respectively).

Most victims of police lethality were killed in the morning and in the afternoon, and it demonstrates that the police ostensibly uses deadly force when most citizens are on the streets, working and studying.

---

O primeiro gráfico que plotei diz respeito ao horário do dia no qual as mortes decorrentes de uso letal da força policial ocorreram. 

Os horários nos quais houve mais vítimas da letalidade policial foram de manhã e a tarde, o que demonstra que a polícia utiliza força letal de forma ostensiva em períodos nos quais a maioria dos cidadãos está circulando nas ruas, e que também coincidem com os horários escolares.

In [ ]:
ax1 = sns.countplot(y = 'horário', color = 'c', data = mortes_estado)
ax1.set_xlabel('mortes');

The next plot shows the number of victims between underage people and adults, and as the following ones, is categorized by ethnicity ("brancos" means whites, and "pretos" means black people).

In both age ranges, the number of white victims is clearly lower than black and pardo victims.

---

O gráfico seguinte, já diferenciado por etnia, mostra o nº de vítimas entre maiores e menores de idade. 

Entre os menores, a incidência de vítimas brancas é de menos da metade das pretas e de aproximadamente 1/3 das pardas.

Em relação a maiores de idade, o número de vítimas brancas, que já é alto, é bem inferior ao de pretas e pardas (em relação às pardas, também corresponde a menos de 50%).


In [ ]:
ax2 = sns.countplot(y = 'faixa etária', data = mortes_estado, 
              hue = mortes_estado.cor.sort_values(), palette = 'Blues');
ax2.set_xlabel('mortes');

In [ ]:
mortes_região_cor = mortes_estado.groupby(['região', 'cor']).agg({'cor': ['size']})
mortes_região_cor.columns = mortes_região_cor.columns.map('_'.join)
mortes_região_cor = mortes_região_cor.reset_index()
mortes_região_cor.rename(columns={'cor_size': 'mortes'}, inplace=True)
mortes_região_cor

The following plots show the number of victims of lethal force by region, and the number of pardo and black victims is higher than white ones in every region of the state.

Regions which concentrate most deaths are the Rio's Metro Area (excluding the capital) and the capital's North Side; and the ones with less deaths are the capital's South Side and the inland part of the state.

Even the regions with less victims have a high number of deaths comparing to whole US states, and the number of victims in the outskirts of the capital (Metro Area, North Side and West Side) is really scary.

---

Os gráficos permitem observar que em todas as regiões do estado, a letalidade policial incide mais sobre pardos e pretos do que sobre brancos. 

As regiões nas quais há mais vítimas do uso letal de forças policiais são a região metropolitana da cidade do Rio (excluindo a capital) e a Zona Norte da capital, enquanto a região com menos registros é a Zona Sul da capital, seguida do interior do Estado.

Os números nas regiões com menos incidência de vítimas já são altos se comparados aos de estados inteiros dos EUA, e a quantidade de vítimas da letalidade policial na periferia da cidade do Rio de Janeiro, ou seja, nas zonas Norte, Oeste e na região metropolitana, é realmente assustadora.

In [ ]:
mrcb = sns.catplot(x = 'cor', y = 'mortes', col = 'região', col_wrap = 2, data = mortes_região_cor, 
                   kind = 'bar', palette = 'Blues', height = 4, aspect = .8)

In [ ]:
mortes_ano_cor = mortes_estado.groupby(['ano', 'cor']).agg({'cor': ['size']})
mortes_ano_cor.columns = mortes_ano_cor.columns.map('_'.join)
mortes_ano_cor = mortes_ano_cor.reset_index()
mortes_ano_cor.rename(columns={'cor_size': 'mortes'}, inplace=True)
mortes_ano_cor

The overall number of victims went down from 2009 to 2013, a period that coincides with the implementation of the Police Pacification Units in many favelas, and also with the preparations for the 2013 FIFA Confederations Cup and the 2014 FIFA World Cup.

From 2013, the number of victims per year has increased: it skyrocketed in 2015, specially when talking about pardos and blacks. The number of white victims has increased remarkably in 2017, (but much less than the deaths of pardos and blacks), and became stable between 2018 and 2019.

---

Em relação ao nº de vítimas ao longo dos anos, houve uma queda no nº geral de vítimas entre 2009 e 2013, período que coincide com a implementação das Unidades de Polícia Pacificadora em diversas favelas, e também com os preparativos para a Copa das Confederações de 2013 e a Copa do Mundo de 2014.

A partir de 2013, o nº de vítimas por ano passou a aumentar, tendo disparado em 2015 em relação a pardos e pretos. O nº de vítimas brancas aumentou de forma notável em 2017, mas de forma bem inferior ao de vítimas pardas e pretas, e se estabilizou entre os anos de 2018 e 2019, diferentemente do que aconteceu em relação às outras etnias.

In [ ]:
ax = sns.lineplot(x = 'ano', y = 'mortes', hue = 'cor',
                  data = mortes_ano_cor, palette = 'Blues')

<a id="section-four"></a>
## Conclusion

There's a high number of overall deaths due to lethal force by Rio de Janeiro's state police, but blacks and pardos (mainly at a young age) are the most affected by police lethality. The exaggerated and unprepared police methods are not the only reason for the high number of deaths: there is a racial and geographical discrimination involved in police actions in the Rio de Janeiro state.

---

O nº de vítimas da letalidade policial no Estado do Rio de Janeiro é assustador em relação à todas as etnias observadas, mas a população negra e periférica (e em sua maioria os jovens) é a que mais sofre com o uso letal da força por parte da polícia. Além da predominância de ações exageradas e do despreparo dos agentes de segurança pública do Estado, fica nítido o racismo institucional das polícias em relação à etnia das vítimas e também à diferença de sua letalidade em área de atuação distintas.